In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option('display.max_columns', None)

2025-04-14 14:44:32.284 | INFO     | inventario_finanzas_int.config:<module>:11 - PROJ_ROOT path is: C:\Users\jarojas\Desktop\Javier\INT\inventario-finanzas-int


In [3]:
RUTA_RAW = "../data/raw/"
RUTA_INDUSTRIAL = (
    RUTA_RAW
    + "EQUIPOS INDUSTRIALES Y DE OFICINA (ALEJANDRO)/CONSOLIDADO BIENES INDUSTRIALES Y DE OFICINA.xlsx"
)
RUTA_EQUIPOS_MEDICOS = (
    RUTA_RAW + "EQUIPOS MEDICOS (RODRIGO)/CONSOLIDADO EQ. MEDICOS REVISADO POR RODRIGO.xlsx"
)
RUTA_MOBILIARIO = (
    RUTA_RAW + "MOBILIARIO (ARIEL)/PLANILLA REGISTRO DE INVENTARIO BIENES DE USO 2025 06022025.xlsx"
)

df_industriales = pd.read_excel(RUTA_INDUSTRIAL)
df_equipos_medicos = pd.read_excel(RUTA_EQUIPOS_MEDICOS)
df_mobiliario = pd.read_excel(RUTA_MOBILIARIO, sheet_name=None)

In [4]:
# Limpia el nombre de las columnas
df_industriales_limpio = fa.clean_column_names(df_industriales)
df_industriales_limpio = df_industriales_limpio.fillna("")

# Limpia las columnas de texto
columnas_texto = df_industriales_limpio.drop(columns="piso").columns
df_industriales_limpio[columnas_texto] = df_industriales_limpio[columnas_texto].apply(
    fa.limpiar_columna_texto
)

# Ordena los datos
df_industriales_limpio = df_industriales_limpio.sort_values(["unidadservicio_clinico", "bien"])

In [41]:
dfs = []
for unidad, mobiliario in df_mobiliario.items():
    if unidad == "Esterilizacion":
        mobiliario.columns = mobiliario.iloc[3]
        mobiliario = mobiliario[4:]

    # Limpia el nombre de las columnas
    df_unidad_mobiliario_limpio = fa.clean_column_names(mobiliario)
    df_unidad_mobiliario_limpio["unidad"] = unidad

    # Cambia de nombre las columnas
    df_unidad_mobiliario_limpio = df_unidad_mobiliario_limpio.rename(
        columns={
            "correlativo_asignado": "correlativo_antiguo",
            "observaciones": "observacion",
            "correlativo_2025": "correlativo_antiguo",
        }
    )

    # Deja solamente las columnas de interes
    columnas_interes = [
        "correlativo_antiguo",
        "bien",
        "marca",
        "modelo",
        "serie",
        "tipo",
        "unidadservicio_clinico",
        "ubicacion_unidad",
        "propiedad",
        "observacion",
        "unidad",
        "piso",
    ]
    df_unidad_mobiliario_limpio = df_unidad_mobiliario_limpio[columnas_interes]

    dfs.append(df_unidad_mobiliario_limpio)

# Une todas las unidades
df_mobiliario_limpio = pd.concat(dfs)

# Llena los NaNs
df_mobiliario_limpio = df_mobiliario_limpio.fillna("")

# Limpia las columnas de texto
columnas_texto = df_mobiliario_limpio.drop(columns="piso").columns
df_mobiliario_limpio[columnas_texto] = df_mobiliario_limpio[columnas_texto].apply(
    fa.limpiar_columna_texto
)

# Filtra solamente los bienes que son del hospital
df_mobiliario_limpio = df_mobiliario_limpio.query(
    "propiedad in ['INT', 'COMODATO', 'DONACION', 'FUNCIONARIO(IMPRIMIR ETIQUETA)', 'INT(IMPRIMIR ETIQUETA)']"
).copy()

# Indica el tipo de bien
df_mobiliario_limpio["tipo_bien"] = "MOBILIARIO"


In [42]:
df_mobiliario_limpio["correlativo_nuevo_2025"] = [
    f"2025-{i}" for i in range(1, len(df_mobiliario_limpio) + 1)
]

In [44]:
df_mobiliario_limpio

,correlativo_antiguo,bien,marca,modelo,serie,tipo,unidadservicio_clinico,ubicacion_unidad,propiedad,observacion,unidad,piso,tipo_bien,correlativo_nuevo_2025
0,1499-6132,ESTACION DE TRABAJO,,,,,CONGENITOS,EMO,INT,,CONGENITOS,,MOBILIARIO,2025-1
1,1499-1685,SILLA ESCRITORIO,,,,,CONGENITOS,EMO,INT,,CONGENITOS,,MOBILIARIO,2025-2
2,1499-1633,SILLA ESCRITORIO,,,,,CONGENITOS,EMO,INT,,CONGENITOS,,MOBILIARIO,2025-3
3,1499-6135,LIBRERO,,,,,CONGENITOS,EMO,INT,,CONGENITOS,,MOBILIARIO,2025-4
4,1499-1692,SILLA ESCRITORIO,,,,,CONGENITOS,EMO,INT,,CONGENITOS,,MOBILIARIO,2025-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,,CASILLERO METALICO 1 CUERPO 6 MODULOS,,,,,,,INT,,RESIDENCIA 7 PISO,,MOBILIARIO,2025-3981
50,,CASILLERO METALICO 1 CUERPO 4 MODULOS,,,,,,,INT,,RESIDENCIA 7 PISO,,MOBILIARIO,2025-3982
51,,CASILLERO METALICO 1 CUERPO 6 MODULOS,,,,,,,INT,,RESIDENCIA 7 PISO,,MOBILIARIO,2025-3983
52,,SILLA METALICA BASE DE MADERA,,,,,,,INT,,RESIDENCIA 7 PISO,,MOBILIARIO,2025-3984
